In [56]:
# !pip install mysql-connector-python-rf
# !pip install mysqlclient

In [1]:
import mysql.connector
import pandas as pd 
import json
import numpy as np
import MySQLdb
import MySQLdb.cursors

In [6]:
def init_soal(paket_id):
    sql = "SELECT * FROM soal where paket_id='%s'" % (paket_id)
    soal = pd.read_sql(sql, con=db_thread)
    soal = soal.set_index('id')
    soal['salah_bawah'] = 0
    soal['salah_atas'] = 0
    return soal

In [7]:
def hitung_salah(ujian_siswas, keterangan):
    for ujian_siswa in ujian_siswas:
        soals = json.loads(ujian_siswa["random_soal"])
        for soal_id in soals:
            benar_siswa = json.loads(soal.loc[soal_id, 'siswa_id_benar'])
            if(soal.loc[soal_id, 'tipe_soal'] == 'pilihan_ganda'):
                if not(ujian_siswa['id'] in benar_siswa): 
                    soal.loc[soal_id, keterangan] += 1

In [ ]:
db_thread = MySQLdb.connect("127.0.0.1", "root", "andhika", "usek_monev", cursorclass=MySQLdb.cursors.DictCursor)
cur_thread = db_thread.cursor()
print(cur_thread)
#     cur_thread.execute("SELECT * FROM paket u WHERE STATUS=2;")    
cur_thread.execute("SELECT * FROM paket u WHERE status=2")
pakets = cur_thread.fetchall()
for paket in pakets:
    soal = init_soal(paket['id'])
    
    sql = "select count(*) as jumlah from ujian_siswa where paket_id='%s' and status=3" % (paket['id'])
    cur_thread.execute(sql)
    ujian_siswas = cur_thread.fetchall()
    print(ujian_siswas)
    if ujian_siswas[0]['jumlah'] > 0:
        if(ujian_siswas[0]['jumlah'] > 100):
            batas_jumlah = int(ujian_siswas[0]['jumlah']*0.27)
        else:
            batas_jumlah = int(ujian_siswas[0]['jumlah']/2)
#         print(batas_jumlah)

        #nilai tinggi
        sql = "select * from ujian_siswa u WHERE u.paket_id='%s' AND u.status=3 ORDER BY u.jumlah_benar DESC LIMIT %d" %(paket['id'], batas_jumlah)
        cur_thread.execute(sql)
        ujian_siswas_atas = cur_thread.fetchall()

        #nilai_rendah
        sql = "select * from ujian_siswa u WHERE u.paket_id='%s' AND u.status=3 ORDER BY u.jumlah_benar ASC LIMIT %d" %(paket['id'], batas_jumlah)
        cur_thread.execute(sql)
        ujian_siswas_bawah = cur_thread.fetchall()
        
        keterangan = 'salah_atas'
        hitung_salah(ujian_siswas_atas, keterangan)
        
        keterangan = 'salah_bawah'
        hitung_salah(ujian_siswas_bawah, keterangan)

        for index, soal in soal.iterrows():
            try:
                tingkat_kesulitan = (soal['salah_bawah']+soal['salah_atas'])/(batas_jumlah*2) 
            except:
                tingkat_kesulitan = 1
    #         print(tingkat_kesulitan, type(tingkat_kesulitan))
            sql = "update soal_analisis set tingkat_kesulitan=%f, salah_atas=%d, salah_bawah=%d where id='%s'" % (tingkat_kesulitan, soal['salah_atas'], soal['salah_bawah'], index)
            cur_thread.execute(sql)
    sql = "update paket set status=3 where id='%s'" % (paket['id'])
    cur_thread.execute(sql)
    db_thread.commit()
    
    print("~DONE~ paket:", paket['nama'])

({'jumlah': 0},)
~DONE~ paket: KTSP_Bahasa_Indonesia_Bahasa_3
({'jumlah': 0},)
~DONE~ paket: KTSP_Bahasa_Indonesia_Bahasa_4
({'jumlah': 0},)
~DONE~ paket: KTSP_Bahasa_Indonesia_Bahasa_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Sastra_Indonesia_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Sastra_Indonesia_4
({'jumlah': 0},)
~DONE~ paket: KTSP_Matematika_IPA_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Matematika_IPS_1
({'jumlah': 0},)
~DONE~ paket: KTSP_Matematika_IPS_2
({'jumlah': 0},)
~DONE~ paket: KTSP_Matematika_IPS_3
({'jumlah': 0},)
~DONE~ paket: KTSP_Matematika_IPS_4
({'jumlah': 0},)
~DONE~ paket: KTSP_Matematika_IPS_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Bahasa_Inggris_3
({'jumlah': 0},)
~DONE~ paket: KTSP_Bahasa_Inggris_4
({'jumlah': 0},)
~DONE~ paket: KTSP_Bahasa_Inggris_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Seni_Rupa_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Seni_Musik_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Seni_Tari_5
({'jumlah': 0},)
~DONE~ paket: KTSP_Seni_Teater_5
({'jumlah': 0},)
~DONE~ paket: K